### Find the qualified workers (criteria: correctly answer at least 4 out of 6 questions)
In this notebook, we checked the answers of AMT workers.


In [4]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

# add path
import os 
import sys 
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

# main
from utils.path import dir_HugeFiles
from utils.preprocessing import load, preprocessing
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv
from utils.word2vec import train_wordvec
from sync_google.access_gsheet import save_to_google, amt, load_google_df
from statsmodels.stats.inter_rater import fleiss_kappa

import pandas as pd
import numpy as np
import seaborn as sns

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.34 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 1.26 s


In [10]:
standard_ans = {'1':['E'],'2':['C'],'3':['B'],'4':['A'], '5':['A','B'], '6': ['D','E']}

time: 21.7 ms


we release the qualification assessments in two batches

In [6]:
def qualification(df):
    completed_worker = df.groupby('WorkerId')['HITId'].count()==6
    completed_worker = completed_worker[completed_worker==True].index

    temp = df[['WorkerId','Input.id','Answer.Your answer.label']].to_dict(orient = 'index')
    new_dict = {}
    for i,v in temp.items():
        if v['WorkerId'] in completed_worker:
            correct_ans = standard_ans[v['Input.id'][12]]
            v['compare'] = (v['Answer.Your answer.label'] in correct_ans)
            new_dict[i] = v

    temp_df = pd.DataFrame.from_dict(new_dict,orient = 'index').sort_values(['WorkerId'])
    stats = temp_df.groupby('WorkerId').sum().unstack()
    goodworker = stats[stats>=4]
    print('%d / %d passed' % (len(goodworker), len(completed_worker)))
    return goodworker, completed_worker

time: 24 ms


In [7]:
qualification(load_google_df(amt['qualify'],'batch1'))

loading
8 / 16 passed


(         WorkerId      
 compare  A1DHL4XO3VT7TR    6.0
          A1NBHCFZKJGHKW    5.0
          A1SX8IVV82M0LW    6.0
          A1Z9JTGIP4A03J    5.0
          A37WDOIQH6JM6V    4.0
          A3JBAUWN5T7E0Q    4.0
          A3LL096CAY5WHB    4.0
          A6FE2ZQNFW12V     4.0
 dtype: float64,
 Index(['A1CA46R2A6TV9W', 'A1DHL4XO3VT7TR', 'A1DXFMUVK5015A', 'A1MHQSP1POQNXX',
        'A1NBHCFZKJGHKW', 'A1SX8IVV82M0LW', 'A1V9MN88VQ1LEB', 'A1YSW3SQ2LYDB2',
        'A1Z9JTGIP4A03J', 'A2FV8JJVCJYNS7', 'A2YRIFGU82QHW', 'A31VO6LWUEE9F9',
        'A37WDOIQH6JM6V', 'A3JBAUWN5T7E0Q', 'A3LL096CAY5WHB', 'A6FE2ZQNFW12V'],
       dtype='object', name='WorkerId'))

time: 2.07 s


In [8]:
qualification(load_google_df(amt['qualify'],'batch2'))

loading
12 / 25 passed


(         WorkerId      
 compare  A1FQGVP8SX5WE2    4.0
          A1KFTQRBMXSC13    5.0
          A1VE1CAMPSYRHE    4.0
          A1YXQDZYCKILTT    4.0
          A1ZD8RU6YB0VEU    4.0
          A2G7F16RAOFTXG    5.0
          A2M1CVZZJAN4T4    5.0
          A3ES33ODBJVKO3    5.0
          A3UUH3632AI3ZX    4.0
          ANFWGSQ8BQRZ      4.0
          APKZZ03P89DHY     4.0
          AQORJW4W8U9YU     4.0
 dtype: float64,
 Index(['A14OPFM8OFA4WF', 'A1FQGVP8SX5WE2', 'A1FS8SBR4SDWYG', 'A1G15BBIUNNN0B',
        'A1KFTQRBMXSC13', 'A1RQGFLPAFX8IB', 'A1VE1CAMPSYRHE', 'A1YXQDZYCKILTT',
        'A1ZD8RU6YB0VEU', 'A2ENGC389KC9U2', 'A2G7F16RAOFTXG', 'A2M1CVZZJAN4T4',
        'A2P065E9CYMYJL', 'A326O3VWP8TO8N', 'A3ES33ODBJVKO3', 'A3UUH3632AI3ZX',
        'A3UZW1NTOX6I15', 'AFQKLIYNCVZTL', 'AFU00NU09CFXE', 'AHT8HA3JRWVGV',
        'AJ2BOHOPWZQNE', 'ANFWGSQ8BQRZ', 'APKZZ03P89DHY', 'AQORJW4W8U9YU',
        'AZLZA0Q87TJZO'],
       dtype='object', name='WorkerId'))

time: 2.48 s


### overall, 16+25 workers completed the task, and only 12+8 passed the task and invited to our experiment